In [5]:
from estimate.vbm import VBM
from residual.residual import Residual
from utils.pihelper import PIHelper
from utils.transform import TransformData
from utils.model import Model
from utils.db import insert_to_table_runtime
from utils.threshold import get_actual_low, get_actual_high, get_sensor_id
from utils.threshold import get_residual_positive_threshold, get_residual_negative_threshold

from db.base import session
from db.diagnostic.models.model_fault import Fault
from db.assets.models.model_assets import Assets
from db.assets.models.model_asset_failure import AssetFailure
from db.diagnostic.models.model_model_tag import ModelTag
from db.diagnostic.models.model_priority import Priority
from db.diagnostic.models.model_diagnose_date import DiagnoseDate

from osisoft.pidevclub.piwebapi.models import PIAnalysis, PIItemsStreamValues, PIStreamValues, PITimedValue, PIRequest

import pandas as pd
import time
from datetime import timedelta, date, datetime

In [6]:
# connect client
pihelper = PIHelper()
client = pihelper.connect_client()

# setup paths
parent = "af:\\\\pi1\\SLA5."
sensor = ['Generator Gross Capacity',
          'Turbine Lube Oil Cooler Outlet Temperature',
          'Turbine.Bearing 1 Drain Oil Temperature',
          'Turbine.Bearing 1 Metal Temperature',
          'Turbine.Bearing 1X Vibration',
          'Turbine.Bearing 1Y Vibration']

paths = [parent+sensor[i] for i in range(len(sensor))]

In [33]:
paths = ['af:\\\\PI1\SLA5.Turbine.Bearing 1 recommendation prescriptive prediction',
         'af:\\\\PI1\SLA5.Turbine.Bearing 1 priority prescriptive prediction',
         'af:\\\\PI1\SLA5.Turbine.Bearing 1 fault prescriptive prediction']

start_time = '2020-01-01 00:00:00'
end_time = '2020-01-05 12:00:00'
interval = '1m'

data = client.data.get_multiple_interpolated_values(paths, start_time=start_time, end_time=end_time, interval=interval)
data

,Value1,Timestamp1,UnitsAbbreviation1,Good1,Questionable1,Substituted1,Value2,Timestamp2,UnitsAbbreviation2,Good2,Questionable2,Substituted2,Value3,Timestamp3,UnitsAbbreviation3,Good3,Questionable3,Substituted3
0,"{'Name': 'No Data', 'Value': 248, 'IsSystem': ...",2019-12-31T17:00:00Z,,False,False,False,"{'Name': 'No Data', 'Value': 248, 'IsSystem': ...",2019-12-31T17:00:00Z,,False,False,False,"{'Name': 'No Data', 'Value': 248, 'IsSystem': ...",2019-12-31T17:00:00Z,,False,False,False
1,"{'Name': 'No Data', 'Value': 248, 'IsSystem': ...",2019-12-31T17:01:00Z,,False,False,False,"{'Name': 'No Data', 'Value': 248, 'IsSystem': ...",2019-12-31T17:01:00Z,,False,False,False,"{'Name': 'No Data', 'Value': 248, 'IsSystem': ...",2019-12-31T17:01:00Z,,False,False,False
2,"{'Name': 'No Data', 'Value': 248, 'IsSystem': ...",2019-12-31T17:02:00Z,,False,False,False,"{'Name': 'No Data', 'Value': 248, 'IsSystem': ...",2019-12-31T17:02:00Z,,False,False,False,"{'Name': 'No Data', 'Value': 248, 'IsSystem': ...",2019-12-31T17:02:00Z,,False,False,False
3,"{'Name': 'No Data', 'Value': 248, 'IsSystem': ...",2019-12-31T17:03:00Z,,False,False,False,"{'Name': 'No Data', 'Value': 248, 'IsSystem': ...",2019-12-31T17:03:00Z,,False,False,False,"{'Name': 'No Data', 'Value': 248, 'IsSystem': ...",2019-12-31T17:03:00Z,,False,False,False
4,"{'Name': 'No Data', 'Value': 248, 'IsSystem': ...",2019-12-31T17:04:00Z,,False,False,False,"{'Name': 'No Data', 'Value': 248, 'IsSystem': ...",2019-12-31T17:04:00Z,,False,False,False,"{'Name': 'No Data', 'Value': 248, 'IsSystem': ...",2019-12-31T17:04:00Z,,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6476,Check bearing cooling system that indicates be...,2020-01-05T04:56:00Z,,True,False,False,3.0,2020-01-05T04:56:00Z,,True,False,False,Local Bearing,2020-01-05T04:56:00Z,,True,False,False
6477,Check bearing cooling system that indicates be...,2020-01-05T04:57:00Z,,True,False,False,3.0,2020-01-05T04:57:00Z,,True,False,False,Local Bearing,2020-01-05T04:57:00Z,,True,False,False
6478,Check bearing cooling system that indicates be...,2020-01-05T04:58:00Z,,True,False,False,3.0,2020-01-05T04:58:00Z,,True,False,False,Local Bearing,2020-01-05T04:58:00Z,,True,False,False
6479,Check bearing cooling system that indicates be...,2020-01-05T04:59:00Z,,True,False,False,3.0,2020-01-05T04:59:00Z,,True,False,False,Local Bearing,2020-01-05T04:59:00Z,,True,False,False
